In [1]:
import mdtraj as mdt
import pprint

In [2]:
def print_class_dir(class_instance):
    """ Prints the dir of the given object but only the
        public methods and attributes.
    """
    public_method_list = [method for method in dir(class_instance) if not method.startswith("_")]
    pprint.pprint(public_method_list)

In [3]:
topology = mdt.Topology()
print("Topology")
print_class_dir(topology)

Topology
['add_atom',
 'add_bond',
 'add_chain',
 'add_residue',
 'atom',
 'atoms',
 'atoms_by_name',
 'bonds',
 'chain',
 'chains',
 'copy',
 'create_disulfide_bonds',
 'create_standard_bonds',
 'delete_atom_by_index',
 'find_molecules',
 'from_dataframe',
 'from_openmm',
 'guess_anchor_molecules',
 'insert_atom',
 'join',
 'n_atoms',
 'n_bonds',
 'n_chains',
 'n_residues',
 'residue',
 'residues',
 'select',
 'select_atom_indices',
 'select_expression',
 'select_pairs',
 'subset',
 'to_bondgraph',
 'to_dataframe',
 'to_fasta',
 'to_openmm']


In [4]:
chain = topology.add_chain()
print("Chain")
print_class_dir(chain)

Chain
['atom',
 'atoms',
 'atoms_by_name',
 'index',
 'n_atoms',
 'n_residues',
 'residue',
 'residues',
 'topology']


In [5]:
print("Residue")
residue = topology.add_residue(name="ALA", chain=chain)
print_class_dir(residue)

Residue
['atom',
 'atoms',
 'atoms_by_name',
 'chain',
 'code',
 'index',
 'is_nucleic',
 'is_protein',
 'is_water',
 'n_atoms',
 'name',
 'resSeq',
 'segment_id']


In [6]:
print(f"Code {residue.code}")
print(f"Name {residue.name}")
print(f"Chain is of type {type(residue.chain)}")

Code A
Name ALA
Chain is of type <class 'mdtraj.core.topology.Chain'>


In [7]:
element = mdt.element.carbon
atom = topology.add_atom(name="C", element=element, residue=residue)
print("Atom")
print_class_dir(atom)

Atom
['element',
 'index',
 'is_backbone',
 'is_sidechain',
 'n_bonds',
 'name',
 'residue',
 'segment_id',
 'serial']


In [8]:
print(atom.residue)

ALA0


In [9]:
print("Element:")
print_class_dir(element)

Element:
['atomic_number',
 'count',
 'getByAtomicNumber',
 'getByMass',
 'getBySymbol',
 'index',
 'mass',
 'name',
 'number',
 'radius',
 'symbol']


In [10]:
element = mdt.element.hydrogen
atom = topology.add_atom(name="H", element=element, residue=residue)
bond = topology.add_bond(
    topology.atom(0),
    topology.atom(1)
)

In [11]:
bonds = list(topology.bonds)
bonds

[Bond(ALA0-C, ALA0-H)]

In [12]:
bonds[0].atom1.residue == bonds[0].atom2.residue

True

In [13]:
print_class_dir(bonds[0])

['atom1', 'atom2', 'count', 'index', 'order', 'type']


In [14]:
print(bonds[0].type)
print(bonds[0].order)

None
None


In [15]:
df, bonds = topology.to_dataframe()
df

,serial,name,element,resSeq,resName,chainID,segmentID
0,None,C,C,0,ALA,0,
1,None,H,H,0,ALA,0,


## Load pdb files

In [16]:
traj_1sux = mdt.load("data/1sux.pdb")
traj_alanine = mdt.load("data/alanine.pdb")

In [17]:
topology_alanine = traj_alanine.topology
print(f"Alanine has {topology_alanine.n_atoms} atoms with {topology_alanine.n_bonds} bonds and {topology_alanine.n_residues} residues")

Alanine has 13 atoms with 12 bonds and 1 residues


In [18]:
df_alanine, bonds_alanine = topology_alanine.to_dataframe()
print(bonds_alanine.shape)

(12, 4)


In [19]:
alanine_bond_list = topology_alanine.bonds
pprint.pprint(list(alanine_bond_list))

[Bond(ALA1-C1, ALA1-C2),
 Bond(ALA1-C1, ALA1-H),
 Bond(ALA1-C1, ALA1-H2),
 Bond(ALA1-C1, ALA1-H3),
 Bond(ALA1-C2, ALA1-C3),
 Bond(ALA1-C2, ALA1-N1),
 Bond(ALA1-C2, ALA1-H4),
 Bond(ALA1-C3, ALA1-O),
 Bond(ALA1-C3, ALA1-OXT),
 Bond(ALA1-OXT, ALA1-H5),
 Bond(ALA1-N1, ALA1-H6),
 Bond(ALA1-N1, ALA1-H7)]


In [20]:
all([b.atom1.residue == b.atom2.residue for b in alanine_bond_list])

True

In [21]:
topology_1sux = traj_1sux.topology
print(f"1SUX has {topology_1sux.n_atoms} atoms with {topology_1sux.n_bonds} bonds")

1SUX has 4244 atoms with 3936 bonds


In [22]:
df_1sux, bonds_1sux = topology_1sux.to_dataframe()
print(bonds_1sux.shape)

(3936, 4)


In [23]:
def is_bond_iterator_ordered_by_residue(topology_):

    residue_index = []
    bonds_iterator = topology_.bonds

    prev_index = next(bonds_iterator).atom1.residue.index
    residue_index.append(prev_index)

    for bond_ in bonds_iterator:
        current_index = bond_.atom1.residue.index
        if current_index != prev_index:
            residue_index.append(current_index)
        prev_index = current_index

    # Now check it the given list is a sequence
    prev_index = residue_index[0]
    for index in residue_index[1:]:
        if prev_index >= index:
            return False

    return True


In [24]:
is_bond_iterator_ordered_by_residue(topology_1sux)

True